### Structured Outputs
Models can be requested to provide their response in a format matching a given schema. This is usefull for ensuring the output can b easily parsed and used in subsequent processing. Langchain supports multiple schema types andmethods for enforcing structured output.
1. Pydantic
2. Typedict
3. Dataclass

#### Pydantic
Pydantic model provides the richest feature set with field validation, description and nested structures.

In [1]:
import os
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

In [2]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The rating of the movie")

In [3]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGoogleGenerativeAI(profile={'max_input_tokens': 1048576, 'max_output_tokens': 65536, 'image_inputs': True, 'audio_inputs': True, 'pdf_inputs': True, 'video_inputs': True, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'image_tool_message': True, 'tool_choice': True}, google_api_key=SecretStr('**********'), model='gemini-2.5-flash', client=<google.genai.client.Client object at 0x0000022B63C6A3F0>, default_metadata=(), model_kwargs={}), kwargs={'response_mime_type': 'application/json', 'response_json_schema': {'properties': {'title': {'description': 'The title of the movie', 'title': 'Title', 'type': 'string'}, 'year': {'description': 'The year the movie was released', 'title': 'Year', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'title': 'Director', 'type': 'string'}, 'rating': {'description': 'The rating 

In [6]:
model.invoke("Provide the details about the movie Interstellar.")


AIMessage(content='"Interstellar" is an epic science fiction film directed by Christopher Nolan, released in 2014. It\'s renowned for its ambitious scope, stunning visuals, thought-provoking themes, and a powerful musical score.\n\nHere are the details:\n\n1.  **Title:** Interstellar\n2.  **Director:** Christopher Nolan\n3.  **Release Date:** November 7, 2014\n4.  **Genre:** Science Fiction, Drama, Adventure, Epic\n5.  **Runtime:** Approximately 169 minutes (2 hours 49 minutes)\n6.  **Music by:** Hans Zimmer (whose score is highly acclaimed and integral to the film\'s atmosphere)\n7.  **Screenplay by:** Jonathan Nolan and Christopher Nolan\n8.  **Starring:**\n    *   **Matthew McConaughey** as Cooper (a former pilot turned farmer)\n    *   **Anne Hathaway** as Dr. Amelia Brand (a scientist on the mission)\n    *   **Jessica Chastain** as Murph (Cooper\'s daughter)\n    *   **Michael Caine** as Professor Brand (Amelia\'s father, the mission\'s leader)\n    *   **Casey Affleck** as Tom (

In [7]:
model_with_structure.invoke("Provide the details about the movie Interstellar.")

Movie(title='Interstellar', year=2014, director='Christopher Nolan', rating=8.7)

In [9]:
#Message output alongside Parsed structure.

from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The rating of the movie")

model_with_structure=model.with_structured_output(Movie, include_raw=True)

response=model_with_structure.invoke("Provide the details about the movie Interstellar.")
response

{'raw': AIMessage(content='{"title":"Interstellar","year":2014,"director":"Christopher Nolan","rating":8.6}', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019bdaf7-606d-7f12-8c08-ed381784ed9f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 10, 'output_tokens': 69, 'total_tokens': 79, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 45}}),
 'parsed': Movie(title='Interstellar', year=2014, director='Christopher Nolan', rating=8.6),
 'parsing_error': None}

In [11]:
#Nested structured output

from pydantic import BaseModel,Field

class Actor(BaseModel):
    name:str=Field(description="The name of the actor")
    role:str=Field(description="The role of the actor")


class MovieDetails(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    cast:list[Actor]=Field(description="The cast of the movie")  #Nested
    genre:str=Field(description="The genre of the movie")
    budget:float | None=Field(None, description="The budget of the movie in crores")

model_with_structure=model.with_structured_output(MovieDetails)

response=model_with_structure.invoke("Provide the details about the movie Interstellar.")
response


MovieDetails(title='Interstellar', year=2014, director='Christopher Nolan', cast=[Actor(name='Matthew McConaughey', role='Cooper'), Actor(name='Anne Hathaway', role='Brand'), Actor(name='Jessica Chastain', role='Murph'), Actor(name='Michael Caine', role='Professor Brand')], genre='Science Fiction', budget=1369.5)

#### TypedDict:
It provides a simpler alternative using Python's built-in typing, idel when you don't need any runtime validation. 

In [12]:
from typing_extensions import TypedDict, Annotated

In [15]:
class MovieDict(TypedDict):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The rating of the movie")


In [16]:
model_with_typedict=model.with_structured_output(MovieDict)

response=model_with_typedict.invoke("Provide the details about the movie Martian.")
response

{'title': 'The Martian',
 'year': 2015,
 'director': 'Ridley Scott',
 'rating': 8.0}

In [17]:
#Nested structured output

from typing_extensions import TypedDict, Annotated

class Actor(TypedDict):
    name:str=Field(description="The name of the actor")
    role:str=Field(description="The role of the actor")


class MovieDetails(TypedDict):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="The year the movie was released")
    director:str=Field(description="The director of the movie")
    cast:list[Actor]=Field(description="The cast of the movie")  #Nested
    genre:str=Field(description="The genre of the movie")
    budget:float | None=Field(None, description="The budget of the movie in crores")

model_with_structure=model.with_structured_output(MovieDetails)

response=model_with_structure.invoke("Provide the details about the movie Martian.")
response

{'title': 'The Martian',
 'year': 2015,
 'director': 'Ridley Scott',
 'cast': [{'name': 'Matt Damon', 'role': 'Mark Watney'},
  {'name': 'Jessica Chastain', 'role': 'Commander Melissa Lewis'},
  {'name': 'Kristen Wiig', 'role': 'Annie Montrose'},
  {'name': 'Jeff Daniels', 'role': 'Teddy Sanders'},
  {'name': 'Michael Peña', 'role': 'Rick Martinez'},
  {'name': 'Sean Bean', 'role': 'Mitch Henderson'},
  {'name': 'Kate Mara', 'role': 'Beth Johanssen'},
  {'name': 'Sebastian Stan', 'role': 'Chris Beck'},
  {'name': 'Aksel Hennie', 'role': 'Alex Vogel'},
  {'name': 'Chiwetel Ejiofor', 'role': 'Vincent Kapoor'}],
 'genre': 'Science Fiction',
 'budget': 108000000}

In [18]:
model.profile

{'max_input_tokens': 1048576,
 'max_output_tokens': 65536,
 'image_inputs': True,
 'audio_inputs': True,
 'pdf_inputs': True,
 'video_inputs': True,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': True,
 'tool_calling': True,
 'structured_output': True,
 'image_url_inputs': True,
 'image_tool_message': True,
 'tool_choice': True}

#### DataClasses
A data class is a class typically containing mainly data, although there aren't really any restrictions. You create in using the @dataclass decorator.

In [ ]:
#Using Pydantic

import os
from langchain.agents import create_agent
from langchain_google_genai import ChatGoogleGenerativeAI
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
)

from pydantic import BaseModel,Field
from langchain.agents import create_agent

class ContactInfo(BaseModel):
    name:str=Field(description="The name of the person")
    phone_number:str=Field(description="The phone number of the person")
    email:str=Field(description="The email of the person")

agent=create_agent(
    model=model,
    response_format=ContactInfo #Auto selects ProviderStratergy.
    
)

result=agent.invoke({
    "messages":[{"role":"user", "content":"Extract contact information from: Sam Altman, samman@openai.com, +1 555 555 5555"}]
    })
result["structured_response"]

ContactInfo(name='Sam Altman', phone_number='+1 555 555 5555', email='samman@openai.com')

In [ ]:
#Using TypedDict

from typing_extensions import TypedDict, Annotated
from langchain.agents import create_agent
class ContactInfo(TypedDict):
    name:str=Field(description="The name of the person")
    phone_number:str=Field(description="The phone number of the person")
    email:str=Field(description="The email of the person")

agent=create_agent(
    model=model,
    response_format=ContactInfo #Auto selects ProviderStratergy.
    
)

result=agent.invoke({
    "messages":[{"role":"user", "content":"Extract contact information from: Sam Altman, samman@openai.com, +1 555 555 5555"}]
    })
result["structured_response"]

{'name': 'Sam Altman',
 'phone_number': '+1 555 555 5555',
 'email': 'samman@openai.com'}

In [ ]:
#Using Dataclass

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    name:str=Field(description="The name of the person")
    phone_number:str=Field(description="The phone number of the person")
    email:str=Field(description="The email of the person")

agent=create_agent(
    model=model,
    response_format=ContactInfo #Auto selects ProviderStratergy.
    
)

result=agent.invoke({
    "messages":[{"role":"user", "content":"Extract contact information from: Sam Altman, samman@openai.com, +1 555 555 5555"}]
    })
result["structured_response"]

ContactInfo(name='Sam Altman', phone_number='+1 555 555 5555', email='samman@openai.com')